For this first trial we assume a naive model (LGM) defined as:
$$dx_t = \sigma_t dW_t^{\mathit{N}}$$

Let's define the Numeraire as:
$$N(t, x_t) = \frac{1}{B(0,t)}exp^{H_tx_t + \frac{1}{2}H_t^2\zeta_t}$$
where $H_t$ and $\zeta_t$ are known functions.

With this let's defined the fundamental equation for the pricing of a derivative under the model. The NPV (Net Present Value) is:
$$V_t = V(t, x_t)$$ 
and the deflated version 
$$\overline{V}_t = V(t, x_t) / N(t, x_t)$$

#### Montecarlo simulation

* Brownian path:
$$W_t \sim \mathcal{N}(0,t)$$
$$W[0] = X_0$$
$$W[t] = W[t - 1]  + \mathcal{Z} \cdot \Delta t^{\frac{1}{2}}$$
with 
$$\mathcal{Z} \sim \mathcal{N}(0,1)$$
* X:
$$X_{t + 1} = X_t + \sigma \cdot (W_{t + 1} - W_t)$$

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import math
from operator import itemgetter
from scipy import stats

class MCSimulation():
    def __init__(self, T, N, X0, sigma, model = 'LGM'):
        self._params = {
            'T':T,
            'N':N,
            'X0':X0,
            'sigma':sigma,
        }
        self._model = model
        
    def simulate(self, nsim = 1e3, show = False):
        T, N, X0, sigma = itemgetter('T', 'N', 'X0', 'sigma')(self._params)
        nsim = int(nsim)
        dt = T / N
        # Brownian simulation
        W, X = np.zeros([N,nsim]), np.zeros([N,nsim])
        # Starting point
        W[0] = X0
        X[0] = X0
        for i in range(1, N):
            W[i] = W[i - 1] + np.random.randn(nsim) * np.sqrt(dt)
        # X simulation
        for i in range(1, N):
            X[i] = X[i - 1] + sigma * (W[i] - W[i - 1])
        if show:
            X = np.linspace(0, T, N)
        
        return X, W

In [3]:
# Strike value
Vt = 2
T = 1
# Set of parameters
T, N, X0, sigma = (T, 100, 0, 0.0075)
mcsimulator = MCSimulation(T, N, X0, sigma)
nsims = int(1e3)
mc_paths, W = mcsimulator.simulate(nsims)
mc_paths_flatten = mc_paths.flatten('C')
deltaTs = np.linspace(0, T, N)
deltaTs = np.tile(deltaTs.reshape(N, 1), nsims).flatten()
df_x = pd.DataFrame(zip(
    deltaTs,
    mc_paths_flatten
), columns = ['dt', 'xt'])

ValueError: too many values to unpack (expected 2)

In [115]:
deltaTs = np.linspace(0, 2, 24)
deltaTs

array([0.        , 0.08695652, 0.17391304, 0.26086957, 0.34782609,
       0.43478261, 0.52173913, 0.60869565, 0.69565217, 0.7826087 ,
       0.86956522, 0.95652174, 1.04347826, 1.13043478, 1.2173913 ,
       1.30434783, 1.39130435, 1.47826087, 1.56521739, 1.65217391,
       1.73913043, 1.82608696, 1.91304348, 2.        ])

#### Visualization

In [104]:
mc_paths_transpose = mc_paths.T
deltaTs = np.linspace(0, T, N)
mc_value_per_time_step = np.mean(mc_paths_transpose, axis = 0)

In [105]:
if nsims < 101:
    plt.figure(figsize = (15,6))
    plt.title('Complete set of paths')
    for vect in mc_paths_transpose:
        sns.lineplot(x = deltaTs, y = vect)
    plt.show()

#### Sanity with zero bond coupon

Numeraire: 

$$N(t, x_t) = \frac{1}{D(t)}exp^{H_tx_t + \frac{1}{2}H_t^2\zeta_t}$$
where:
* $D(t)$ given and follows = $D(t) = e^{-rt}$, in this example with $r = 0.03$
* $H(t) = \frac{1 - e^{-\kappa t}}{\kappa}$, with $\kappa = 2$

Discount factor:

$$Z(x_t, t, T) = \frac{D(T)}{D(t)}exp-((H_T - H_t)x_t - \frac{1}{2}(H_T^2-H_t^2)\zeta_t) = \frac{D(T)}{D(t)}exp(-(H_T - H_t)x_t - \frac{1}{2}H_T^2\zeta_t+\frac{1}{2}H_t^2\zeta_t) = $$
$$\frac{D(T)}{D(t)}exp(-H_Tx_t + H_tx_t - \frac{1}{2}H_T^2\zeta_t+\frac{1}{2}H_t^2\zeta_t) = \frac{D(T)}{D(t)}exp(-\frac{1}{2}H_T^2\zeta_t-H_Tx_t)exp(H_tx_t + \frac{1}{2}H_t^2\zeta_t) = $$
$$D(T)exp(-\frac{1}{2}H_T^2\zeta_t-H_Tx_t)N(t, x_t)$$
where:
* $\zeta(t) = \int_0^t\sigma^2(s)ds$, with $\sigma(s)$ a piecewise function.

The sanity aims to retrieve the $D(t)$ after aggregating for each time step $t$ on the previous simulations. Steps:
* Calculate $Z(\cdot)$ for each timestep
* Calculate the numeraire $N(\cdot)$
* Get the $\hat{D}(t)$ for each path and time step as $\hat{D}(t) = \frac{Z(\cdot)}{N(\cdot)} \to E[\hat{D}(t)] = D(t)E[exp(-\frac{1}{2}H_T^2\zeta_t-H_Tx_t)] = D(T)exp(E[-\frac{1}{2}H_T^2\zeta_t-H_Tx_t]) = D(T)$
* Aggregate and compare the value with the theoretical $D(t)$

The final objective is to check that $E[-\frac{1}{2}H_T^2\zeta_t-H_Tx_t] = 0$

In [106]:
# Discount factor
def D(t, r = 0.03):
    return np.exp(-r * t)
# H
def H(t, kappa = 2):
    return (1 - np.exp(-kappa * t)) / kappa
# Volatility function
def sigma(t, sigma_0 = 0.75):
    return sigma_0 + (t // 2) * 0.05
# Zeta
def C(t, sigma = sigma):
    import scipy.integrate as integrate
    return integrate.quad(lambda x: sigma(x)**2, 0, t)[0]    
# Numeraire
def N(t, xt, ct):
    return 1/D(t) * np.exp(H(t) * xt + 0.5 * H(t)**2 * ct)
# Factor de descuento a tiempo t dado
def Z(xt, t, T, ct = None):
    assert ct is not None
    return D(T) * np.exp(-0.5 * H(T)**2 * ct - H(T)*xt) * N(t, xt, ct)

In [111]:
df_x.sort_values(['dt']).head(10)

,dt,xt,ct,d_hat_t
0,0.0,0.0,0.0,0.941765
658,0.0,0.0,0.0,0.941765
659,0.0,0.0,0.0,0.941765
660,0.0,0.0,0.0,0.941765
661,0.0,0.0,0.0,0.941765
662,0.0,0.0,0.0,0.941765
663,0.0,0.0,0.0,0.941765
664,0.0,0.0,0.0,0.941765
665,0.0,0.0,0.0,0.941765
666,0.0,0.0,0.0,0.941765


In [112]:
D(T) * np.exp(-0.5 * H(T) * df_x.ct.values - H(T) * df_x.xt)

0        0.941765
1        0.941765
2        0.941765
3        0.941765
4        0.941765
           ...   
23995    0.714399
23996    0.703209
23997    0.727597
23998    0.717490
23999    0.711322
Name: xt, Length: 24000, dtype: float64

In [109]:
t_unique = df_x.dt.unique()
dict_C = {dt:C(dt) for dt in t_unique}
df_x['ct'] = df_x.apply(lambda x: dict_C[x['dt']], axis = 1)
xt, t, T, ct = df_x.xt, df_x.dt, T, df_x.ct
df_x['d_hat_t'] = Z(xt, t, T, ct) / N(t, xt, ct)
sanity = df_x.groupby(['dt']).agg(
    d_hat_t = ('d_hat_t', 'mean')
).reset_index()
sanity['dts'] = D(np.array(sorted(t_unique)))

In [110]:
sanity.head(120)

,dt,d_hat_t,dts
0,0.000000,0.941765,1.000000
1,0.086957,0.936234,0.997395
2,0.173913,0.930666,0.994796
3,0.260870,0.925173,0.992204
4,0.347826,0.919584,0.989619
5,0.434783,0.914244,0.987041
6,0.521739,0.908836,0.984470
7,0.608696,0.903426,0.981905
8,0.695652,0.898161,0.979347
9,0.782609,0.893036,0.976795


#### Seq2seq with feed forward neural networks

Check:
* https://towardsdatascience.com/how-to-use-custom-losses-with-custom-gradients-in-tensorflow-with-keras-e87f19d13bd5
* https://www.tensorflow.org/guide/autodiff

The idea is to include in the loss function the gradient tape to respect the model!!

In [130]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional

In [185]:
keras.backend.clear_session() 
keras.backend.set_floatx('float64')

In [279]:
class LGM_model(tf.keras.Model):
    
    def __init__(
        self,
        n_steps,
        intermediate_dim=64,
        is_sequential = False,
        name="LGM_NN_model",
        **kwargs
    ):
        super(LGM_model, self).__init__(name=name, **kwargs)
        input_layer = keras.Input(shape=(n_steps,), name='input_nn')
        x = layers.Layer(trainable = False, name = 'adhoc_structure_layer')(input_layer)
        num_layer = 0
        if is_sequential:
            x = layers.GRU(intermediate_dim, name = 'sequential_layer')(x)
            num_layer += 1
        output_layer = layers.Dense(units = n_steps, activation = 'relu', name = 'first_dense')(x)
        self._custom_model = model = keras.Model(
            inputs=[input_layer],
            outputs=[output_layer],
        )
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")
        self._grads = None

    @property
    def metrics(self):
        return [self.loss_tracker]
    
    @property
    def model(self):
        return self._custom_model

    # Function to get dV/dX after each epoch
    # TODO: Change into a function that returns the grads per variable and not all the grads.
    def get_dv_dx(self, features):
        xs = tf.Variable(features)
        with tf.GradientTape() as tape:
            y = self._custom_model(xs)
        # This represents dV/dX
        self._grads = tape.gradient(y, xs)
        return self._grads
    
    def get_dv_dxi(self, i, sample_idx = 0):
        print(i)
        return self._grads[sample_idx][i] if self._grads is not None else None

    def call(self, inputs):
        return self._custom_model(inputs)
    
    # Path prediction
    # TODO: Vectorize for simultaneous multiple paths
    # TODO: Check
    def predict_path(self, x, sample_idx = 0):
        # Steps
        N = x.shape[1]
        # X path
        x_path = x[sample_idx]
        # Swaping option at strike
        v = np.zeros((1,N))[0]
        predictions = self._custom_model(x).numpy()[0]
        # Keep only the first value predicted
        v[0] = predictions[0]
        # Get the gradients
        grads = self.get_dv_dx(x).numpy()[sample_idx]
        # Do the iterative process
        for i in range(0, N - 1):
            v[i + 1] = v[i] + grads[i] * (x_path[i + 1] - x_path[i])
        
        return v, predictions

#### Iterative process
* F - neural network function.
* $\frac{\delta F}{\delta X_t}^i$ - gradient calculated by using the model at $i$-iteration.
* $\phi(n, x_n)$ - known terminal function.


**Path generation**
$$\hat{V} = F(X)$$

$$\overline{V}_0 = \hat{V}^i[0]$$

$$\overline{V}_{t+1} = \overline{V}_t + \frac{\delta F(X)}{\delta x_t}(x_{t + 1} - x_{t})$$

**Loss function**
$$\mathcal{L}(\overline{V}, \hat{V}) = \beta_1 \cdot (\hat{V}_n - \phi(n, x_n))^2 + \beta_2\cdot (\hat{V}_n - \frac{\delta F(X)}{\delta x_n})^2 + \sum_{i = 1}^{n - 1}(\overline{V}_i - \hat{V}_i)^2  $$

In [300]:
@tf.function
def f(xn, n):
  return xn ** 2 + n

In [301]:
# TODO: Change np.arrays to tf.Tensor
# TODO2: Include better method descriptions
def custom_loss_lgm(x = np.array, path = np.array, predictions = np.array):
    ''' 
    Beta:
        * Beta[0] - error related to predictions
        * Beta[1] - error related to strike
        * Beta[2] - error related to derivative at strike
    '''
    betas = [1.0, 0.5, 0.5]
    # Careful: Using global variable...
    len_path = N
    # For f and f'
    xn = tf.Variable(x[0,-1], name = 'xn')
    n = tf.Variable(np.float64(len_path), name = 'n')
    n_idx = int(len_path)
    # Loss given the strike function
    strike_loss = (x[-1] - f(xn, n))**2
    # Autodiff f
    with tf.GradientTape() as tape:
        y = f(xn, n)
    grad_df = tape.gradient(y, {
        'xn':xn,
        'n':n    
    })
    df_dxn = grad_df['xn']
    # Careful: global variable
    derivative_loss = (lgm.get_dv_dxi(n_idx - 1) - df_dxn)**2
    # Epoch error per step
    error_per_step = np.sum((path[:-1] - predictions[:-1])**2)

    return np.dot(np.array(betas), np.array([error_per_step, strike_loss, derivative_loss]))

In [304]:
epochs = 2
# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
# Custom model
lgm = LGM_model(N, 64)
# Data used as features
features = np.reshape(mc_value_per_time_step, (1, N))
# Loss metric
loss_metric = tf.keras.metrics.Mean()
# Iterate over epochs.
for epoch in range(epochs):
    print("Start of epoch %d" % (epoch,))
    
    with tf.GradientTape() as tape:
        x = tf.Variable(features, trainable = False)
        path = tf.Variable(path, trainable = False)
        predictions = tf.Variable(predictions, trainable = False)        
        path, predictions = lgm.predict_path(features)
        loss = custom_loss_lgm(x = features,  path = path, predictions = predictions)

    grads = tape.gradient(loss, lgm.trainable_weights)
    optimizer.apply_gradients(zip(grads, lgm.trainable_weights))

    loss_metric(loss)
    print("Mean loss = %.4f" % (loss_metric.result()))

Start of epoch 0
99
Mean loss = 4999.9395
Start of epoch 1
99
Mean loss = 4999.9167


c:\Users\borja\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
